In [1]:
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Lambda
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.engine.training import _make_batches as make_batches
# from keras.models import Model
from keras.utils import plot_model



from flipGradientTF import GradientReversal
from loader import load_3a_data, load_chk_data
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from sklearn.model_selection import StratifiedKFold

/home/owen/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 10
nb_epoch = 30
_TRAIN = K.variable(1, dtype='uint8')

# Read data from college and senior high data sets
### Source Data: College, a.k.a., 3a
### Target Data: Senior high, a.k.a., chk

In [3]:
SCALE = True
OVERSAMPLING = False

xs_train_1a, ys_train_1a = load_3a_data('ncu_data_week_1-6(1a).csv', oversampling=OVERSAMPLING, scale=SCALE)
xs_train_2a, ys_train_2a = load_3a_data('ncu_data_week_1-12(2a).csv', oversampling=OVERSAMPLING, scale=SCALE)
xs_train_3a, ys_train_3a = load_3a_data('ncu_data_week_1-18(3a).csv', oversampling=OVERSAMPLING, scale=SCALE)
xs_train_2d, ys_train_2d = load_3a_data('ncu_data_week_7-12(2d).csv', oversampling=OVERSAMPLING, scale=SCALE)
xs_train_3d, ys_train_3d = load_3a_data('ncu_data_week_13-18(3d).csv', oversampling=OVERSAMPLING, scale=SCALE)

xs_train = np.concatenate((xs_train_1a, xs_train_2a, xs_train_3a, xs_train_2d, xs_train_3d), axis=0)
ys_train_3a = np.concatenate((ys_train_1a, ys_train_2a, ys_train_3a, ys_train_2d, ys_train_3d), axis=0)
ys_train = to_categorical(ys_train_3a)

xt_train_chka, yt_train_chka = load_chk_data('ncu_data_week_chka_1-6.csv', oversampling=OVERSAMPLING, scale=SCALE)
xt_train_chkb, yt_train_chkb = load_chk_data('ncu_data_week_chkb_1-6.csv', oversampling=OVERSAMPLING, scale=SCALE)
xt_train = np.concatenate((xt_train_chka, xt_train_chkb),axis=0)
yt_train_chk = np.concatenate((yt_train_chka, yt_train_chkb),axis=0)

xt_train = xt_train[range(0, len(ys_train_3a))]
yt_train = to_categorical(yt_train_chk[range(0, len(ys_train_3a))])

print("length of each datasets:", "source: ", len(ys_train), "target: ", len(yt_train))

length of each datasets: source:  295 target:  295


In [4]:
# Create domain classification labels
domain_labels = np.vstack([np.tile([0, 1], [int(batch_size / 2), 1]),
                           np.tile([1., 0.], [int(batch_size / 2), 1])])

# Generate models, batches, and plot

In [5]:
def batch_gen(batches, id_array, data, labels):
    for batch_index, (batch_start, batch_end) in enumerate(batches):
        batch_ids = id_array[batch_start:batch_end]
        if labels is not None:
            yield data[batch_ids], labels[batch_ids]
        else:
            yield data[batch_ids]
        np.random.shuffle(id_array)
        
def imshow_grid(images, shape=[2, 8]):
    """Plot images in a grid of a given shape."""
    fig = plt.figure()
    grid = ImageGrid(fig, 111, nrows_ncols=shape, axes_pad=0.05)

    size = shape[0] * shape[1]
    for i in range(size):
        grid[i].axis('off')
        # The AxesGrid object work as a list of axes.
        grid[i].imshow(np.swapaxes(np.swapaxes(images[i], 0, 2), 0, 1))
        
def plot_embedding(X, y, d, title=None):
    """Plot an embedding X with the class label y colored by the domain d."""
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    # Plot colors numbers
    plt.figure(figsize=(10, 10))
    plt.subplot(111)
    for i in range(X.shape[0]):
        # plot colored number
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.bwr(d[i] / 1.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)


In [6]:
class DANNBuilder(object):
    def __init__(self):
        self.model = None
        self.net = None
        self.domain_invariant_features = None
        self.grl = None
        self.opt = SGD()

    def _build_feature_extractor(self, model_input):
        '''Build segment of net for feature extraction.'''
        net = Dense(units=19, activation='relu', input_dim=19)(model_input)
        self.domain_invariant_features = net
        return net

    def _build_classifier(self, model_input):
        net = Dense(38, activation='relu')(model_input)
        net = Dropout(0.1)(net)
        net = Dense(units=2, activation='softmax', name='classifier_output')(model_input)
        return net

    def build_source_model(self, main_input, plot_model=False):
        net = self._build_feature_extractor(main_input)
        net = self._build_classifier(net)
        model = Model(inputs=main_input, outputs=net)
        model.compile(loss={'classifier_output': 'categorical_crossentropy'},
                      optimizer=self.opt, metrics=['accuracy'])
        return model

    def build_dann_model(self, main_input, plot_model=False):
        net = self._build_feature_extractor(main_input)
        self.grl = GradientReversal(1.0)
        branch = self.grl(net)
        branch = Dense(128, activation='relu')(branch)
        branch = Dropout(0.1)(branch)
        branch = Dense(2, activation='relu', name='domain_output')(branch)

        # When building DANN model, route first half of batch (source examples)
        # to domain classifier, and route full batch (half source, half target)
        # to the domain classifier.
        # 先前面一半的 data 拿去 domain classifier
#         net = Lambda(lambda x: K.switch(K.learning_phase(),
#                      x[:int(batch_size / 2), :], x),
#                      output_shape=lambda x: ((batch_size / 2,) +
#                      x[1:]) if _TRAIN else x[0:])(net)

        net = self._build_classifier(net)
        #model for source data
        model_src = Model(inputs=main_input, outputs=[branch, net])
        #model for target data
        model_tgt = Model(inputs=main_input, outputs=[branch])

        model_src.compile(loss={'classifier_output': 'categorical_crossentropy',
                      'domain_output': 'categorical_crossentropy'},
                      optimizer=self.opt, metrics=['accuracy'])

        model_tgt.compile(loss={'domain_output': 'categorical_crossentropy'},
                      optimizer=self.opt, metrics=['accuracy'] )
#         model = Model(input=main_input, output=[branch, net])
#         if plot_model:
#             plot(model, show_shapes=True)
#         model.compile(loss='categorical_crossentropy',
#                       optimizer=self.opt, metrics=['accuracy'])
        return (model_src, model_tgt)

    def build_tsne_model(self, main_input):
        '''Create model to output intermediate layer
        activations to visualize domain invariant features'''
        tsne_model = Model(inputs=main_input,
                           outputs=self.domain_invariant_features)
        return tsne_model

In [7]:
main_input = Input(shape=(19,), name='main_input')
builder = DANNBuilder()

# Training Source (3a)

In [8]:
src_model = builder.build_source_model(main_input)
#print('Training source only model')
src_model.fit(xs_train, ys_train, batch_size=batch_size, epochs=nb_epoch, verbose=0)

plot_model(src_model, to_file='model.png')

source_only_accuracy = src_model.evaluate(xs_train, ys_train)[1]
print('Source(3a) Only Accuracy: ', source_only_accuracy)

source_model_on_target_accuracy = src_model.evaluate(xt_train, yt_train)[1]
print('Source(3a) to Target Accuracy: ', source_model_on_target_accuracy)

295/295 [==============================] - 0s 218us/step
Source(3a) Only Accuracy:  0.7796610175553015
295/295 [==============================] - 0s 54us/step
Source(3a) to Target Accuracy:  0.7016949154562869


# Training Target (chk)

In [9]:

src_model = builder.build_source_model(main_input)
print('Training target(chk) only model')
src_model.fit(xt_train, yt_train, batch_size=batch_size, epochs=nb_epoch, verbose=0)

target_only_accuracy = src_model.evaluate(xt_train, yt_train)[1]
print('Target(chk) Only Accuracy: ', target_only_accuracy)

target_model_on_source_accuracy = src_model.evaluate(xs_train, ys_train)[1]
print('Target(chk) to Source Accuracy: ', target_model_on_source_accuracy)

Training target(chk) only model
295/295 [==============================] - 0s 233us/step
Target(chk) Only Accuracy:  0.976271186440678
295/295 [==============================] - 0s 49us/step
Target(chk) to Source Accuracy:  0.46779661027051633


In [10]:
# Broken out training loop for a DANN model.
source_index_arr = np.arange(xs_train.shape[0])
target_index_arr = np.arange(xt_train.shape[0])

#nb_epoch = nb_epoch * 10

batches_per_epoch = len(xs_train) / batch_size
num_steps = nb_epoch * batches_per_epoch

# initialize DANN model
#main_input = Input(shape=(19,), name='main_input')
#builder = DANNBuilder()
dann_src_model, dann_tgt_model = builder.build_dann_model(main_input)

plot_model(dann_src_model, to_file='dann_src_model.png')

j = 0

print('Training DANN model')

metric_src_epoch_list = []
metric_tgt_epoch_list = []

for i in range(nb_epoch):

    source_batches = make_batches(xs_train.shape[0], int(batch_size / 2))
    target_batches = make_batches(xt_train.shape[0], int(batch_size / 2))

    source_gen = batch_gen(source_batches, source_index_arr, xs_train, ys_train)
    target_gen = batch_gen(target_batches, target_index_arr, xt_train, None)

    losses = list()
    acc = list()

    if i % (nb_epoch / 5) == 0:
        #print 'loss: %f, domain accuracy: %f, class accuracy: %f' % (loss, da, pa)
        print('Epoch ', i, source_gen)
    
    metric_src_batch_list = []
    metric_tgt_batch_list = []

    for (xs_batch, ys_batch) in source_gen:

        # Update learning rate and gradient multiplier as described in the paper.
        p = float(j) / num_steps
        l = 2. / (1. + np.exp(-10. * p)) - 1
        lr = 0.01 / (1. + 10 * p)**0.75
        builder.grl.l = l
        builder.opt.lr = lr

        #print('xb.shape[0]', xb.shape[0])
        if xs_batch.shape[0] != batch_size / 2:
            continue

        try:
            #print('try')
            xt_batch = next(target_gen)
        except:
            # Regeneration
            target_gen = batch_gen(target_batches, target_index_arr, xt_train, None)

        metric_src = dann_src_model.train_on_batch({'main_input': xs_batch},
                                            {'classifier_output': ys_batch,
                                            'domain_output': domain_labels[:batch_size//2]},
                                            )
        
        metric_src_batch_list.append(metric_src)
        
        metric_tgt = dann_tgt_model.train_on_batch({'main_input': xt_batch},
                                            {'domain_output': domain_labels[batch_size//2:]},
                                            )
        
        metric_tgt_batch_list.append(metric_tgt)
        j += 1
        
    metric_src_mean = np.mean(np.asarray(metric_src_batch_list), axis=0)
    metric_tgt_mean = np.mean(np.asarray(metric_tgt_batch_list), axis=0)

    metric_src_epoch_list.append(metric_src_mean)
    metric_tgt_epoch_list.append(metric_tgt_mean)
    #print("tgt_metric", metric_tgt_mean,"\nsrc_metric", metric_src_mean)
    
    

/home/owen/.local/lib/python3.5/site-packages/keras/engine/topology.py:638: UserWarning: Class `flipGradientTF.GradientReversal` defines `get_output_shape_for` but does not override `compute_output_shape`. If this is a Keras 1 layer, please implement `compute_output_shape` to support Keras 2.
  output_shape = self.compute_output_shape(input_shape)


Training DANN model
Epoch  0 <generator object batch_gen at 0x7f0ea0704fc0>
Epoch  6 <generator object batch_gen at 0x7f0ea0704fc0>
Epoch  12 <generator object batch_gen at 0x7f0ea0704fc0>
Epoch  18 <generator object batch_gen at 0x7f0ea0704fc0>
Epoch  24 <generator object batch_gen at 0x7f0ea0704fc0>


In [11]:
print('DANN model is training by xs_train, ys_train, xt_train')

print('Evaluating source samples on DANN model')
dann_src_model_out = dann_src_model.predict_on_batch(xs_train)
dann_src_model_classifier_out = np.argmax(dann_src_model_out[1], axis=1)
dann_src_model_classifier_out_actual = np.argmax(ys_train, axis=1)
dann_to_source_acc = float(np.sum((dann_src_model_classifier_out == dann_src_model_classifier_out_actual))) / float(len(dann_src_model_classifier_out))
print('DANN to Source Accuracy(chk): ', dann_to_source_acc)

print('Evaluating target samples on DANN model')
dann_tgt_model_out = dann_src_model.predict_on_batch(xt_train)
dann_tgt_model_classifier_out = np.argmax(dann_tgt_model_out[1], axis=1)
dann_tgt_model_classifier_out_actual = np.argmax(yt_train, axis=1)
dann_to_target_acc = float(np.sum((dann_tgt_model_classifier_out == dann_tgt_model_classifier_out_actual))) / float(len(dann_tgt_model_classifier_out))
print('DANN to Target Accuracy(3a): ', dann_to_target_acc)

print('Evaluating domain classification')
dann_tgt_model_out = dann_tgt_model.predict_on_batch(np.concatenate((xs_train, xt_train), axis=0))
dann_tgt_model_domain_out = np.argmax(dann_tgt_model_out, axis=1)
dann_tgt_model_domain_out_actual = np.argmax(np.vstack([
    np.tile([0, 1], [len(xs_train), 1]),
    np.tile([1., 0.], [len(xt_train), 1])
]), axis=1)
domain_classification_acc = float(np.sum((dann_tgt_model_domain_out == dann_tgt_model_domain_out_actual))) / float(len(dann_tgt_model_domain_out))
print('Domain Accuracy: ', domain_classification_acc)

DANN model is training by xs_train, ys_train, xt_train
Evaluating source samples on DANN model
DANN to Source Accuracy(chk):  0.3559322033898305
Evaluating target samples on DANN model
DANN to Target Accuracy(3a):  0.8169491525423729
Evaluating domain classification
Domain Accuracy:  0.5


In [12]:
print ("source only accuracy:", source_only_accuracy)
print ("dann on target:", dann_to_target_acc)
print ("dann on target accuracy should between", source_model_on_target_accuracy, "and", target_only_accuracy)

source only accuracy: 0.7796610175553015
dann on target: 0.8169491525423729
dann on target accuracy should between 0.7016949154562869 and 0.976271186440678


In [13]:
# imshow_grid(x_train)
# imshow_grid(xt_train)
src_vis = builder.build_tsne_model(main_input)
dann_vis = builder.build_tsne_model(main_input)
# Created mixed dataset for TSNE visualization
# num_test = 500
combined_test_imgs = np.vstack([xs_train[:], xt_train[:]])
combined_test_labels = np.vstack([ys_train[:], yt_train[:]])
combined_test_domain = np.vstack([np.tile([1., 0.], [len(xs_train), 1]),
                                 np.tile([0., 1.], [len(xt_train), 1])])

src_embedding = src_vis.predict([combined_test_imgs])
src_tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=300)
tsne = src_tsne.fit_transform(src_embedding)

plot_embedding(tsne, combined_test_labels.argmax(1),
               combined_test_domain.argmax(1), 'Source only')

dann_embedding = dann_vis.predict([combined_test_imgs])
dann_tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=300)
tsne = dann_tsne.fit_transform(dann_embedding)

plot_embedding(tsne, combined_test_labels.argmax(1),
               combined_test_domain.argmax(1), 'DANN')

#plt.show()
# pltOut(os.path.join(run_name,"tsne.png"))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').